In [204]:
import random
import cn2an
def generate_index() -> int:
    return random.randint(1, 60)
def random_element(list) -> str:
    return list[random.randint(0, len(list) - 1)]
def unicode():
    val = random.randint(0x4e00, 0x9fbf)
    return chr(val)
def gbk2312():
    head = random.randint(0xb0, 0xf7)
    body = random.randint(0xa1, 0xfe)
    val = f'{head:x} {body:x}'
    str = bytes.fromhex(val).decode('gb2312')
    return str

In [205]:
weight_decorator = [
    ["%0%", 12],
    ["请%0%", 4],
    ["帮我%0%", 4],
    ["给我%0%", 4]
]
template_decorator = []
for [template, weight] in weight_decorator:
    template_decorator.extend([template] * weight)
def decorate_template(template) -> str:
    return random_element(template_decorator).replace("%0%", template)

In [206]:
template_click = [
    "点击第%0%个",
    "查看第%0%个",
    "进入第%0%个",
    "第%0%个点击",
    "按一下第%0%个",
    "打开第%0%个",
    "看下第%0%个",
    "查看第%0%个",
]
def generate_click() -> [str, str]:
    vindex = cn2an.an2cn(generate_index())
    output = '{"command":"click","vindex":"%s","mode":0}' % (vindex)
    template = random_element(template_click)
    input = decorate_template(template.replace("%0%", '%s' % (vindex)))
    return [input, output]

In [207]:
template_swipe = [
    "第%0%个%1%滑",
    "向%1%滑动第%0%个",
    "%1%滑第%0%个",
    "把第%0%个向%1%滑",
    "第%0%个往%1%",
    "向%1%移动第%0%个"
]
direction_swipe = [
    "左",
    "右",
    "上",
    "下"
]
def generate_swipe() -> [str, str]:
    vindex = cn2an.an2cn(generate_index())
    direction = random_element(direction_swipe)
    output = '{"command":"swipe","vindex":"%s","direction":"%s"}' % (vindex, direction)
    template = random_element(template_swipe)
    input = decorate_template(template.replace("%0%", '%s' % (vindex)).replace("%1%", direction))
    return [input, output]

In [208]:
import pandas as pd
df = pd.read_csv("./data/train_split.csv")
available_text = df['Utterance']

In [209]:
template_input_text0 = [
    "向第%0%个中输入%1%",
    "输入%1%到第%0%个",
    "向第%0%个尾部添加%1%",
    "添加%1%到第%0%个",
    "向第%0%个尾部输入%1%"
]
def generate_input_text_0() -> [str, str]:
    vindex = cn2an.an2cn(generate_index())
    text = random_element(available_text).replace(" ", "")[0: 20]
    output = '{"command":"input_text","vindex":"%s","text":"%s","mode":0}' % (vindex, text)
    template = random_element(template_input_text0)
    input = decorate_template(template.replace("%0%", '%s' % (vindex)).replace("%1%", text))
    return [input, output]

In [210]:
train_set = []
for i in range(0, 5000):
    train_set.append(generate_click())
for i in range(0, 5000):
    train_set.append(generate_swipe())
for i in range(0, 10000):
    train_set.append(generate_input_text_0())
random.shuffle(train_set)

In [211]:
f = open("./data/train.txt", "w+")
for line in train_set:
    f.write("%s %s\n" % (line[0], line[1]))
f.close()